In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# import data
data=pd.read_csv('D:/lh_data/Churn_Modelling.csv')
data

In [ ]:
data.isnull().sum()

In [ ]:
X=data.iloc[:,3:13]   #dropping the first 3 columns
y=data.iloc[:,13]
print(X.shape, y.shape)

In [ ]:
data.groupby("Geography").Geography.count()

In [ ]:
data.groupby("Gender").Geography.count()

In [ ]:
# dummy1
#categorical_columns = ['Geography', 'Gender']
#for column in categorical_columns:
#    tempdf = pd.get_dummies(X[column], prefix=column)
#    X = pd.concat((X,tempdf), axis=1)
#    X = X.drop(columns=column)
#X.head()

In [ ]:
# dummy2
geography=pd.get_dummies(X['Geography'],prefix='Geography_')
gender=pd.get_dummies(X['Gender'],prefix='Gender_',drop_first=True)

In [ ]:
X=pd.concat([X,geography,gender],axis=1)
X=X.drop(['Geography','Gender'],axis=1)
X.head()

In [ ]:
# split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=0)
print(X.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)


In [ ]:
# scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
# import keras
import keras
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, PReLU, ELU, Dropout

In [ ]:
model_class=Sequential()
model_class.add(Dense(units=8, kernel_initializer='he_uniform', activation='relu', input_dim=12))
model_class.add(Dense(units=8, kernel_initializer='he_uniform', activation='relu'))
model_class.add(Dense(units=1, kernel_initializer='he_uniform', activation='sigmoid'))
model_class.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model_history=model_class.fit(X_train, y_train, validation_split=0.33, batch_size=10, epochs=100)

In [ ]:
print(model_history.history.keys())

In [ ]:
print(model_class.summary())

In [ ]:
# history of loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [ ]:
# history of loss
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [ ]:
# predict
y_pred=model_class.predict(X_test)
y_pred

In [ ]:
y_pred=y_pred>0.5
y_pred

In [ ]:
print(y_test.shape, y_pred.shape)

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

In [ ]:
# accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
score

## Tunning tool
### hyperparameters: HL,Neurons,LR

#!pip install keras-tuner

In [ ]:
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch


In [ ]:
def build_model(hp):
    model=keras.Sequential()
    for i in range(hp.Int('num_layers',2,7)):
        model.add(layers.Dense(units=hp.Int('units_'+str(i),min_value=32,max_value=512,step=32),
                               activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(keras.optimizers.Adam(hp.Choice('learning_rate',values=[0.01,0.001,0.0001])), 
                                        loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
tunning=RandomSearch(build_model, 
                    objective='val_accuracy',
                    max_trials=5,
                    executions_per_trial=3,
                    directory='my_directory',
                    project_name='ANN_001' )

In [ ]:
tunning.search_space_summary()

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)
print(X_train2.shape, X_val.shape, y_train2.shape, y_val.shape)

In [ ]:
tunning.search(X_train2, y_train2, epochs=5, validation_data=(X_val,y_val))

In [ ]:
tunning.results_summary()

In [ ]:
model_class=Sequential()
model_class.add(Dense(units=224, kernel_initializer='he_uniform', activation='relu', input_dim=12))
model_class.add(Dense(units=256, kernel_initializer='he_uniform', activation='relu'))
model_class.add(Dense(units=32, kernel_initializer='he_uniform', activation='relu'))
model_class.add(Dense(units=32, kernel_initializer='he_uniform', activation='relu'))
model_class.add(Dense(units=32, kernel_initializer='he_uniform', activation='relu'))
model_class.add(Dense(units=1, kernel_initializer='he_uniform', activation='sigmoid'))
model_class.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model_history=model_class.fit(X_train, y_train, validation_split=0.3, batch_size=10, epochs=100)

In [ ]:
# history of loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [ ]:
# predict
y_pred=model_class.predict(X_test)
y_pred
y_pred=y_pred>0.5
y_pred

In [ ]:
y_pred = np.squeeze(y_pred)
pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True)


In [ ]:
score=accuracy_score(y_pred,y_test)
score